## Implementing 3 Versions of the EM-Algorithmn in R.
- Classic EM
- Classification EM
- Stochastic EM 


In [1]:

##########################################################

############### Helper Functions #########################

##########################################################


compute_ll <- function(P_Z,data,alphas, mu, sigma, method) {  
  #' #___________________________________________________________________  
  #' Compute the Log-likelihood formula          
  #' EM for observed Data, CEM for Complete Data = Observed + Unobserved  |
  #' #_____________________________________________________________________
  L <- matrix(0, nrow = length(data), ncol = length(alphas))
  
  for (k in 1:length(mu)){
    L[, k] <- alphas[k]*dnorm(data, mu[k], sqrt(sigma[k]))
  }
  
  #print(method)
  return(sum(log (rowSums(L))))  
}




gen_Z <- function(data, alpha, mu, sigma){ 
  
  #' Computes P(Zi=k|Xi) := Z_ij
  
  #' Normalized over all rows  
  
  #' 
  
  L <- matrix(0, nrow = length(data), ncol = length(alpha))
  
  for (k in 1:length(mu)){
    
    L[, k] <- alpha[k]*dnorm(data, mu[k], sqrt(sigma[k]))
    
  }
  
  L <- t((apply(L,MARGIN = 1, function(x) x/sum(x))))
  
  return(L) #ixk Matrix of Posterior Probs
  
}





assign_z <- function(P_Z, method = "EM"){  
  
  
  if (any(is.na(P_Z))){ 

    # This is the case, when bad initial Guesses have been made for example.
    # probabilities which are effectively zero, are a  hint for too many clustercomponents or a very large Variance.     
    
    cat("Iteration aborted: at r: \n")
    
    cat("| colSums of P_Z: ", colSums(P_Z), "|")
    
    print("\n")
    
    return("BREAK") #only breaks this function ...
    
  }
  
  
  
  if (method == "CEM"){
    
    #C-Step:  rounds to maxValue
    
    for (i in 1:nrow(P_Z)){
      
      locator <- which.max(P_Z[i,])
      
      rest <- setdiff(1:ncol(P_Z),locator)
      
      P_Z[i,locator] <- 1
      
      P_Z[i,rest] <-0
      
    }
    
  }
  
  if (method == "SEM"){
    
    #S-Step: Stochastic assignment 
    #P_Z <- apply(P_Z,1,function(x) rmultinom(1,1,x))
    for (i in 1:nrow(P_Z)){
      
      P_Z[i,] <- rmultinom(1,1,P_Z[i,])
      
    }
    
  }
  
  return(P_Z) # Label Matrix 
  
}





est_var <- function(mu, P_Z, data, omit = FALSE){ # correct 
  
  #'Estimates the Variance of r-th iteration
  
  #'for each k
  
  sigmas <- c()
  
  for (k in 1:ncol(P_Z)){
    
    sigmas <- c(sigmas,sum((P_Z[,k]*((data-mu[k])^2))/sum(P_Z[,k])))
    
  }
  
  #sigmas <- replace(sigmas, is.na(sigmas), 1e-2) # avoid zero-division damages
  
  if (omit == TRUE) sigmas <- na.omit(sigmas)
  
  return(sigmas) # returns vector of variances
  
}


In [2]:


##########################################################

############## EM Implementation #########################

##########################################################

#X-vector - mixing_weight - mu- sigma  

EM <- function(data, a, m, s, max_iter = 1000, method = "EM", tol = 1e-2, 
               
               known_a = FALSE, known_s = FALSE){
  
  
  
  ll <- c(Inf) #list Log-Likelihoods
  
  output <- list()
  
  s <- s^2 # either here, or input it right away as variances
  
  
  
  for (r in 1:max_iter){
    
    
    
    #E-Step: Compute Probabilities of Data belonging to K-th component
    
    P_Z <- gen_Z(data, a,m,s) #Posterior Probs: z_ij's #rowSums = 1
    
    ###########################################################
    
    # C or S-Step: assign label-components with a given method#
    
    P_Z <- assign_z(P_Z, method = method)
    
    if (is.character(P_Z)) return() #else its matrix and thus good
    
    ###########################################################
    
    # M-Step: re-estimate all Parameters 
    
    if (known_a == FALSE) a <- apply(P_Z,MARGIN = 2, function(x) mean(x)) # N instead of N_k
    
    m <- apply(P_Z, MARGIN = 2, function(x) sum((x*data)/sum(x))) # x being Z_ij heren
    
    if (known_s == FALSE) s <- est_var(m, P_Z, data, omit = FALSE)
    
    
    
    # Variance != Standard Deviation
    
    
    
    ########################################################################################## 
    
    if (any(is.na(c(a,m,s)))) { # in Case something gets nulled out                           |
      
      cat("Iteration aborted: at r: ", r, "\n")          #                                    |
      
      cat("| colSums of P_Z: ", colSums(P_Z), "|")       #                                    |
      
      print("\n")
      
      return () #output
      
    }
    
    ##########################################################################################
    
    ll <- c(ll,compute_ll(P_Z,data,a,m,s,method = method)) # append r-th Log-Likelihood 
    
    
    
    #MU <- rbind(MU, m)
    
    #SIGMA <- rbind(SIGMA, s)
    
    #ALPHA <- rbind(ALPHA, a)
    
    
    
    if (any(is.na(ll))){
      
      #cat("Iteration aborted: at r: ", r, "\n")
      
      #cat("| colSums of P_Z: ", colSums(P_Z), "|")
      
      #print("\n")
      
      return () #output 
      
    }
    
    
    
    eps <- abs(ll[r]-ll[r+1]) 
    
    if (is.na(eps)) return()  #could fail at first iter, thats why is.na() 
    
    if (eps < tol){ # Change for SEM 
      
      #cat("Converged at",r,"-th Iteration !")
      
      output[["m"]] <- m
      
      output[["a"]] <- a
      
      output[["s"]] <- s
      
      output[["LogLik"]] <- ll[2:r+1]
      
      output[["r"]] <- r
      
      #output[["params"]] <- list("mu"= MU, "sigma"= SIGMA, "alpha"= ALPHA)
      
      if (method != "EM") output[["cluster"]] <- apply(P_Z,1,function(x) which.max(x))
      
      return (output)
      
    }
    
    # Continue the Loop 
    
  } 
  
  #Exit r-loop: If maximum iter is reached without convergence
  
  cat("Failed to converged at",r,"-th Iteration for given tolerance: ", tol, "!")
  
  cat("Final Error:",eps)
  
  output[["final_epsilon"]] <- eps
  
  output[["m"]] <- m
  
  output[["a"]] <- a
  
  output[["s"]] <- s
  
  output[["r"]] <- r
  
  output[["LogLik"]] <- ll[2:r+1]
  
  #output[["params"]] <- list("mu"= MU, "sigma"= SIGMA, "alpha"= ALPHA)
  
  if (method != "EM") output[["cluster"]] <- apply(P_Z,1,function(x) which.max(x))
  
  return (output)
               }

## Monte Carlo Simulations

In [3]:
library(RColorBrewer)
library(viridisLite)
library(dplyr)
library(ggplot2)

set.seed(123)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [4]:
gen_data <- function(n, m, s, a){
  
  #'
  # Generate df with labels 
  
  df <- data.frame("X" = NULL, "component" = NULL)

  for (i in (1:length(m))){
    proportion <- round(n*a[i])
    x <- rnorm(proportion, m[i], s[i]) # ? 
    df <- rbind(df,data.frame("X" = x,"component" = rep(i,proportion)))
  }

  df <- df[sample(1:nrow(df)),] #optional
  
  return(df)
  
}
#
truem <- c(12,18, 26)
trues <- c(2, 3, 6)
truea <- c(0.25,0.4,0.35)

truem2 <- c(10,24, 40)
trues2 <- c(2.6, 3, 6)
truea2 <- c(0.25,0.4,0.35)


In [5]:
simulate <- function(r, truea, truem, trues, off_m = 1,off_s = 1){
  
  df_em <- data.frame("mse_m" = 0,"mse_s" = 0,"mse_a" = 0, "ll" = 0,
                      "iter" = 0, "time" = 0, "size" = 0)
  
  df_cem <- data.frame("mse_m" = 0,"mse_s" = 0,"mse_a" = 0, "ll" = 0,
                       "iter" = 0, "time" = 0,"size" = 0, "clust" = 0)
  
  df_sem <- data.frame("mse_m" = 0,"mse_s" = 0, "mse_a" = 0, "ll" = 0,
                       "iter" = 0, "time" = 0, "size" = 0, "clust" = 0)

  #m_0 <- rnorm(length(truem),truem,off_m)
  #s_0 <- abs(rnorm(length(trues), trues, off_s))
  a_0 <- rep(1/length(truem), length(truem)) # same weights 
  size = 1500
  fail <- c()
  M0 <- c()
  S0 <- c()
  
  for (i in 1:r){
    print(i)
    df1 <- gen_data(n = size , m = truem ,s = trues , a = truea)
    data <- df1$X
    
    # With different Starting positions:
    m_0 <- rnorm(length(truem),mean(data),off_m)
    #m_0 <- c(quantile(data,0.3),quantile(data,0.6),quantile(data,0.9))
    s_0 <- abs(rnorm(length(trues), sd(data), off_s))
    M0 <- rbind(M0,m_0)
    S0 <- rbind(S0,s_0)
    #############################
    ###### simulate EM ##########
    ############################
    
    tem <- system.time(EM1 <- EM(df1$X,a_0, m_0,s_0, max_iter = 500, method ="EM",
              tol = 0.01))
    
    if (is.null(EM1)){
      df_em <- rbind(df_em,rep(NA, 7))
      fail <- c(fail, "EM")
    }else{
      df_em <- rbind(df_em,c(sum((  ord(EM1$m,truem)  - truem)^2),
                             sum((  ord(sqrt(EM1$s), trues)   - trues)^2),
                             sum((  ord(EM1$a, truea)   - truea)^2),
                             max(EM1$LogLik), EM1$r, tem[3], size)) 
    }
    
    #############################
    ###### simulate CEM #########
    ############################
    
    
    tcem <- system.time(CEM1 <- EM(df1$X,a_0, m_0,s_0, max_iter = 500, method ="CEM",
                                  tol = 0.01))
    
    if (is.null(CEM1)){
      fail <- c(fail, "CEM")
      df_cem <- rbind(df_cem, rep(NA, 8))
    }else{
    ccoef <- (sum(df1$component == CEM1$cluster)/length(df1$component))
    df_cem <- rbind(df_cem,c(sum((  ord(CEM1$m,truem)  - truem)^2),
                             sum((  ord(sqrt(CEM1$s),trues)   - trues)^2),
                             sum((  ord(CEM1$a,truea)   -  truea)^2), 
                             max(CEM1$LogLik), CEM1$r, tcem[3], size, ccoef))
    }
    
    #############################
    ###### simulate SEM #########
    #############################
    
    
    tsem <- system.time(SEM1 <- EM(df1$X,a_0, m_0,s_0, max_iter = 500, method ="SEM",
                                    tol = 0.01))
    if (is.null(SEM1)){
      fail <- c(fail, "SEM")
      df_sem <- rbind(df_sem,rep(NA, 8))
    }else{
      ccoef2 <- (sum(df1$component == SEM1$cluster)/length(df1$component))
    df_sem <- rbind(df_sem,c(sum((  ord(SEM1$m, truem)   - truem)^2),
                             sum((  ord(sqrt(SEM1$s), trues)   - trues)^2),
                             sum((  ord(SEM1$a, truea)   - truea)^2),
                             max(SEM1$LogLik), SEM1$r, tsem[3], size, ccoef2))
    }
  }
  print("Done")
  df_em <- df_em[-1,]
  df_cem <- df_cem[-1,]
  df_sem <- df_sem[-1,]
  
  return(list("EM" = df_em, "CEM" = df_cem, "SEM" = df_sem, "m0" = M0, "s0" = S0))
  
}


In [ ]:
######################################################################
#######################   Data 1  ####################################
######################################################################
test <- simulate(100, truea, truem, trues, off_m = 1.96,off_s = 1)###
################################################################
#saveRDS(test, file="test_n.RData")
test <- readRDS("test.RData")

sum(is.na(test$EM$ll))
sum(is.na(test$CEM$ll)) # 9 times
sum(is.na(test$SEM$ll))
#########
mak <- turbo(100)
colem <- mak[100]
colcem <- mak[10]
colsem <- mak[60]

In [ ]:
#################################
######### Boxplots ##############
#################################
par(mfrow = c(1,3))
boxplot(test$EM$iter,test$CEM$iter,test$SEM$iter,col = c(colem,colcem,colsem), 
        xaxt = "n", main = "Number of Iteration", ylab = "n")
axis(1, at=1:3, labels=c("EM", "CEM", "SEM"))


boxplot(test$EM$mse_a,test$CEM$mse_a,test$SEM$mse_a,col = c(colem,colcem,colsem), 
        xaxt = "n", main = "MSE", ylab = expression(MSE*(alpha)))
axis(1, at=1:3, labels=c("EM", "CEM", "SEM"))


boxplot(test$EM$ll,test$CEM$ll,test$SEM$ll, xaxt = "n",col = c(colem,colcem,colsem),
        main = "LogLik", ylab = "LL")
axis(1, at=1:3, labels=c("EM", "CEM", "SEM"))

####### Clustering ##########
par(mfrow = c(1,1))
boxplot(100*test$CEM$clust,100*test$SEM$clust, xaxt = "n",col = c(colcem,colsem),
        main = "Clustercoefficient", ylab = "%")
axis(1, at=1:2, labels=c("CEM", "SEM"))
cat("EM: ",mean(100*test$CEM$clust, na.rm = TRUE), "SEM:",mean(100*test$SEM$clust, na.rm = TRUE))


In [ ]:

#################################
###### Scatterplots #############
#################################
par(mfrow = c(1,1))
min_x <- min(na.omit(c(test$EM$mse_m,test$CEM$mse_m,test$SEM$mse_m)))
max_x <- max(na.omit(c(test$EM$mse_m,test$CEM$mse_m,test$SEM$mse_m)))

min_y <- min(na.omit(c(test$EM$mse_s,test$CEM$mse_s,test$SEM$mse_s)))
max_y <- max(na.omit(c(test$EM$mse_s,test$CEM$mse_s,test$SEM$mse_s)))

########################### MSE ###################################
plot(1, type="n", xlab=expression(MSE(mu)), ylab=expression(MSE(sigma)),
     xlim=c(min_x, max_x), ylim=c(min_y, max_y), main = "MSE - Intricate Mixture")

points(test$EM$mse_m, test$EM$mse_s, col = colem , pch = 20)
abline(v = mean(test$EM$mse_m), h = mean(test$EM$mse_s),  col = colem )
points(mean(test$EM$mse_m),  mean(test$EM$mse_s),  col = "black")

points(test$CEM$mse_m, test$CEM$mse_s, col = colcem , pch = 18)
abline(v = mean(test$CEM$mse_m,na.rm = TRUE), h = mean(test$CEM$mse_s, na.rm = TRUE), col = colcem)
points(mean(test$CEM$mse_m,na.rm = TRUE), mean(test$CEM$mse_s, na.rm = TRUE), col = "black")

points(test$SEM$mse_m, test$CEM$mse_s, col = colsem ,pch = 4)
abline(v = mean(test$SEM$mse_m), h = mean(test$SEM$mse_s),col = colsem )
points(mean(test$SEM$mse_m), mean(test$SEM$mse_s),col = "black" )

legend("topright",inset = .02, legend = c("EM", "CEM", "SEM"),
       col=c(colem, colcem, colsem), pch=c(20,18,4), cex=0.8)

##################################################